### ДЗ Дерево решений

In [1]:
from sklearn.datasets import load_iris
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from collections import Counter

e:\Users\lyalina_ia\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Функция для расчета энтропийного коэффициента

In [2]:
def HEntropy(l):
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

Функция для расчета коэффициента Джини

In [3]:
def HGini(l):
    length = len(l)
    cnt = Counter(l)
    
    gini = 0
    for cl in cnt.values():
        p_1 = cl / length
        p_2 = (1 - p_1)
        it = p_1 * p_2
        gini += it
    
    return gini

Функция для расчета критерия информативности

In [4]:
def IG(H, l, i):
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)

Расчет критерия информативности для заданного разбиения переменной

In [5]:
def test_H(H, l):
    #print("{:5} {:3}   {:4} {:4} {:4}".format("#","l","IG","Hl","Hr"))
    #print("-"*24)
    dfIG = []
    for i in range(1,len(l)):

        #print("{:2}. {:3}   {:.2f} {:.2f} {:.2f}".format(i, l.iloc[i], IG(H, l, i), H(l[:i]), H(l[i:])))
        dfIG.append([i,IG(H, l, i)])

    return dfIG

Функция построения дерева решений: 
    по выбранному признаку сортируем датасет, передаем отсортированный массив значений переменной, последовательно проверяем точку разбиения переданного массива. Определяем шаг, при котором коэф. информативности максимальный, для визуализации выводим значения класса при максимальном
    коэф. инф. и значение признака в точке разбиения, по которому проводилась сортировка.

В зависимости от заданного параметра критерия информативности, будем строить дерево по энтропийному критерию и критерию Джини.

In [6]:
def DT_H(coef,y,x):
    step_max_IG = 0
    n = 0

    print("{:5}   {:4}   {:4}  {:4}   {:4}".format("#","IG max", "# элемента", "значение класса",  "значение признака"))
    while step_max_IG < len(y):
            n += 1
            IG_H = test_H(coef, y[step_max_IG:])
            IG_H = pd.DataFrame(IG_H)
            if IG_H[1].idxmax() != 0:
                step_max_IG += IG_H.iloc[IG_H[1].idxmax()][0].astype(int)
            else: 
                step_max_IG =  len(y)
            
            print("Шаг {:2}.   {:.2f}    {:.0f}              {:.2f}        {:.50}".format(
                n, IG_H[1].max(), step_max_IG, y.iloc[step_max_IG-1], x.iloc[step_max_IG-1].to_string() ))

Загрузми данные из набора Ирис для построения дерева решений

In [7]:
iris = load_iris()

In [8]:
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.DataFrame(iris.target, columns=['species'])

Проходим по всем признакам и определяем какой из них дает максимальное значение коэф. информативности на первом шаге построения дерева.

В данном случае    IG max  = 0.92    для признакак      petal width (cm) <   0.6

In [9]:
print('Дерево решений по критерию информативности = коэф. энтропии')
for i  in range(len(X.columns)):
    col = X.columns[i]
    print('  Построение дерева для признака', X.columns[i] )
    #сортировка по текущему признаку
    df = pd.concat([X,y], axis=1)
    df.sort_values(by=[col],inplace=True)
    X = df.drop(['species'],axis = 1)
    y = df['species']
    DT_H(HEntropy,y,X.iloc[:, [i]])
    


Дерево решений по критерию информативности = коэф. энтропии
  Построение дерева для признака sepal length (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.60    55              0.00        sepal length (cm)    5.5
Шаг  2.   0.20    76              1.00        sepal length (cm)    5.8
Шаг  3.   0.12    138              1.00        sepal length (cm)    7.0
Шаг  4.   0.00    150              2.00        sepal length (cm)    7.9
  Построение дерева для признака sepal width (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.34    71              2.00        sepal width (cm)    3.0
Шаг  2.   0.16    113              2.00        sepal width (cm)    3.3
Шаг  3.   0.08    140              2.00        sepal width (cm)    3.8
Шаг  4.   0.00    150              0.00        sepal width (cm)    4.4
  Построение дерева для признака petal length (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.92    50    

Проверим как определяется коэф. для критерия Джини

In [10]:
print('Дерево решений по критерию информативности = коэф. Джини')
for i  in range(len(X.columns)):
    col = X.columns[i]
    print('  Построение дерева для признака', X.columns[i] )
    #сортировка по текущему признаку
    df = pd.concat([X,y], axis=1)
    df.sort_values(by=[col],inplace=True)
    X = df.drop(['species'],axis = 1)
    y = df['species']
    DT_H(HGini,y,X.iloc[:, [i]]) 

Дерево решений по критерию информативности = коэф. Джини
  Построение дерева для признака sepal length (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.24    55              0.00        sepal length (cm)    5.5
Шаг  2.   0.09    99              1.00        sepal length (cm)    6.2
Шаг  3.   0.05    138              1.00        sepal length (cm)    7.0
Шаг  4.   0.00    150              2.00        sepal length (cm)    7.9
  Построение дерева для признака sepal width (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.14    73              2.00        sepal width (cm)    3.0
Шаг  2.   0.07    112              2.00        sepal width (cm)    3.3
Шаг  3.   0.01    123              1.00        sepal width (cm)    3.4
Шаг  4.   0.02    134              0.00        sepal width (cm)    3.6
Шаг  5.   0.09    150              0.00        sepal width (cm)    4.4
  Построение дерева для признака petal length (cm)
#       IG max   # эл

Исключаем из выборки признак полученный на первом шаге, строим дерево для разделения остальных элементов.

В данном случае на втором шаге лучший результат (IG max = 0.66) показывает разбиение по признаку petal length (cm)  <  4.7

In [23]:
print('Дерево решений по критерию информативности = коэф. энтропии')
print('Выбор второго признака для разбиения')

df1 = pd.concat([X,y], axis=1)
X1 = df1.loc[df1['petal width (cm)'] > 0.6]
y1 = X1['species']

X1 = X1.drop(['petal width (cm)','species'],axis = 1)

for i  in range(len(X1.columns)):
    col = X1.columns[i]
    print('  Построение дерева для признака', X1.columns[i] )
    #сортировка по текущему признаку
    df1 = pd.concat([X1,y1], axis=1)
    df1.sort_values(by=[col],inplace=True)
    X1 = df1.drop(['species'],axis = 1)
    y1 = df1['species']
    DT_H(HEntropy,y1,X1.iloc[:, [i]])
    

Дерево решений по критерию информативности = коэф. энтропии
Выбор второго признака для разбиения
  Построение дерева для признака sepal length (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.17    48              1.00        sepal length (cm)    6.2
Шаг  2.   0.12    88              1.00        sepal length (cm)    7.0
Шаг  3.   0.00    100              2.00        sepal length (cm)    7.9
  Построение дерева для признака sepal width (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.08    60              1.00        sepal width (cm)    3.0
Шаг  2.   0.10    67              2.00        sepal width (cm)    3.0
Шаг  3.   0.09    96              1.00        sepal width (cm)    3.4
Шаг  4.   0.00    100              2.00        sepal width (cm)    3.8
  Построение дерева для признака petal length (cm)
#       IG max   # элемента  значение класса   значение признака
Шаг  1.   0.66    45              1.00        petal length (c

Для сравнения визуализируем дерево решения с использованием библиотеки DecisionTreeClassifier

In [12]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=2)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=2)

In [13]:
from sklearn.tree import export_graphviz

def get_tree_dot_view(clf, feature_names=None, class_names=None):
    print(export_graphviz(clf, out_file=None, filled=True, feature_names=feature_names, class_names=class_names))

In [14]:
clf = DecisionTreeClassifier(max_depth=3)
clf.fit(X, y)

DecisionTreeClassifier(max_depth=3)

In [15]:
get_tree_dot_view(clf, list(X.columns), iris.target_names)

digraph Tree {
node [shape=box, style="filled", color="black"] ;
0 [label="petal length (cm) <= 2.45\ngini = 0.667\nsamples = 150\nvalue = [50, 50, 50]\nclass = setosa", fillcolor="#ffffff"] ;
1 [label="gini = 0.0\nsamples = 50\nvalue = [50, 0, 0]\nclass = setosa", fillcolor="#e58139"] ;
0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;
2 [label="petal width (cm) <= 1.75\ngini = 0.5\nsamples = 100\nvalue = [0, 50, 50]\nclass = versicolor", fillcolor="#ffffff"] ;
0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;
3 [label="petal length (cm) <= 4.95\ngini = 0.168\nsamples = 54\nvalue = [0, 49, 5]\nclass = versicolor", fillcolor="#4de88e"] ;
2 -> 3 ;
4 [label="gini = 0.041\nsamples = 48\nvalue = [0, 47, 1]\nclass = versicolor", fillcolor="#3de684"] ;
3 -> 4 ;
5 [label="gini = 0.444\nsamples = 6\nvalue = [0, 2, 4]\nclass = virginica", fillcolor="#c09cf2"] ;
3 -> 5 ;
6 [label="petal length (cm) <= 4.85\ngini = 0.043\nsamples = 46\nvalue = [0, 1, 45]\nclass = virgini

![text](tree.png)